In [2]:
import pandas as pd
import numpy as np
from dotenv import dotenv_values, find_dotenv
import os
from datacleaning.functions import filter_by_granularity
from statsmodels.tsa.api import VAR
# from statsmodels.tsa.stattools import adfuller
# from statsmodels.tools.eval_measures import rmse, aic
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
# set path parameters
config = dotenv_values(find_dotenv())
path_rawdata = os.path.abspath(config["RAWDATA"]) + '//'
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '//'
path_figures = os.path.abspath(config["FIGURES"]) + '//'

# for plots
plt.rcParams.update({'font.size': 25})

In [3]:
# import the industry-by-industry I-O requirements table
requirementstable = pd.read_pickle(path_cleandata + 'requirements_naics6.pkl')
iorequirements_wide_fillna = requirementstable.fillna(value=0)

# import pce data
beadata = pd.read_pickle(path_cleandata + 'BEA_PCE.pkl')
# products at 6th level of granularity
beadata = filter_by_granularity(beadata, target_granularity=6)

In [4]:
# make a nice matrix

# remove nans if any 
iorequirements_wide_fillna = requirementstable.fillna(value=0)

# find sectors that show up as both buyers and sellers (to get a nice square matrix)
iosectors = list(set(iorequirements_wide_fillna.index) & set(iorequirements_wide_fillna.columns))
iorequirements_wide_fillna = iorequirements_wide_fillna[iosectors].loc[iosectors]
# list of buyers and sellers (remove redundant sectors for which total expenses/sales are zero)
inputsectors = [index for index in iorequirements_wide_fillna.index if iorequirements_wide_fillna.sum(axis=1)[index] != 0 and iorequirements_wide_fillna.sum(axis=0)[index] != 0]
outputsectors = [column for column in iorequirements_wide_fillna.columns if iorequirements_wide_fillna.sum(axis=1)[column] != 0 and iorequirements_wide_fillna.sum(axis=0)[column] != 0]
# get the intersection of these lists
sectors_to_include = list(set(inputsectors) & set(outputsectors))

# filter I-O table and residuals based on this list of sectors (to get a nice square matrix)
iorequirements_wide_fillna.drop(columns=[col for col in iorequirements_wide_fillna.columns if col not in sectors_to_include], inplace=True)
iorequirements_wide_fillna.drop(index=[idx for idx in iorequirements_wide_fillna.index if idx not in sectors_to_include], inplace=True)

# sort sector names (honestly not necessary but it keeps me organized)
iorequirements_wide_fillna = iorequirements_wide_fillna.sort_index().reindex(sorted(iorequirements_wide_fillna.columns), axis=1)

In [21]:
requirements_inv = np.linalg.inv(iorequirements_wide_fillna)
requirements_inv = pd.DataFrame(requirements_inv, index=iorequirements_wide_fillna.index, columns=iorequirements_wide_fillna.columns)

requirements_inv_colsum = requirements_inv.sum(axis=0)
requirements_inv_rowsum = requirements_inv.sum(axis=1)

print(requirements_inv_colsum[requirements_inv_colsum >= 1])
print(requirements_inv_rowsum[requirements_inv_rowsum >= 1])

desc_O
Customs duties    1.0
dtype: float64
desc_I
Community food, housing, and other relief services, including vocational rehabilitation services    1.0
Customs duties                                                                                      1.0
Death care services                                                                                 1.0
Educational and vocational structures                                                               1.0
Elementary and secondary schools                                                                    1.0
Health care structures                                                                              1.0
Home health care services                                                                           1.0
Manufacturing structures                                                                            1.0
Multifamily residential structures                                                                  1.0
Office and co

In [22]:
# get the industry-by-industry I-O matrix
iomatrix_wide = np.identity(len(iorequirements_wide_fillna)) - np.linalg.inv(iorequirements_wide_fillna)
iomatrix_wide = pd.DataFrame(iomatrix_wide, index=iorequirements_wide_fillna.index, columns=iorequirements_wide_fillna.columns)

iomatrix_wide_colsum = iomatrix_wide.sum(axis=0)
iomatrix_wide_rowsum = iomatrix_wide.sum(axis=1)

print(iomatrix_wide_colsum[iomatrix_wide_colsum == 0])
print(iomatrix_wide_rowsum[iomatrix_wide_rowsum == 0])

desc_O
Customs duties    0.0
dtype: float64
desc_I
Community food, housing, and other relief services, including vocational rehabilitation services    0.0
Customs duties                                                                                      0.0
Death care services                                                                                 0.0
Educational and vocational structures                                                               0.0
Elementary and secondary schools                                                                    0.0
Health care structures                                                                              0.0
Home health care services                                                                           0.0
Manufacturing structures                                                                            0.0
Multifamily residential structures                                                                  0.0
Office and co

In [7]:
# get a long form I-O matrix (i kinda don't know how to use the concordance otherwise)

iomatrix_long = pd.melt(iomatrix_wide.reset_index(), id_vars=['desc_I'], var_name='desc_O', value_name='value')

# now taking some steps to get a product-by-product version

# import concordance
concordance = pd.read_pickle(path_cleandata + 'concordance//concordance6_naics6.pkl')

# change concordance column names so that i can merge with I-O table (first merge with sellers then merge with buyers)
crosswalk_I = concordance.rename(columns={'product': 'product_I', 'NAICS_desc': 'desc_I'})
crosswalk_O = concordance.rename(columns={'product': 'product_O', 'NAICS_desc': 'desc_O'})

# merge crosswalk to sellers 
add_naics_I = pd.merge(left=crosswalk_I, right=iomatrix_long, on='desc_I', how='inner')[['product_I', 'desc_I', 'desc_O', 'value']]
add_naics_O = pd.merge(left=crosswalk_O, right=iomatrix_long, on='desc_O', how='inner')[['product_O', 'desc_O', 'desc_I', 'value']]

iomatrix_long = pd.merge(left=add_naics_I, right=add_naics_O, on=['desc_I', 'desc_O', 'value'], how='outer')

# collapse by product_O
iomatrix_long_byproduct = iomatrix_long[['product_I', 'desc_I', 'value', 'product_O']].groupby(['product_I', 'desc_I', 'product_O'], as_index=False).sum(min_count=1)

In [8]:
# import make table
maketable_wide = pd.read_pickle(path_cleandata + 'make_naics6.pkl')
maketable_wide = maketable_wide.fillna(value=0)

# get sum of total sales for each industry
sales = maketable_wide.sum(min_count=1, axis=1)

# merge total sales with concordance (start to get by-product shares of sales to use as weights)
sales_byproduct = pd.merge(left=sales.reset_index().rename(columns={0: 'sales'}), right=crosswalk_I, on='desc_I', how='inner')

# get total sales for each product
total_sales_by_product = sales_byproduct.groupby('product_I')['sales'].sum().reset_index()
total_sales_by_product.columns = ['product_I', 'total_sales']
sales_byproduct = pd.merge(left=sales_byproduct, right=total_sales_by_product, how='inner', on='product_I')

# calculate each industry's share within each product category
sales_byproduct['share_of_sales_by_industry'] = sales_byproduct['sales'] / sales_byproduct['total_sales']

In [9]:
# merge this shares column into the uhhhh
iomatrix_long_byproduct = pd.merge(left=iomatrix_long_byproduct, right=sales_byproduct[['desc_I', 'product_I', 'share_of_sales_by_industry']], on=['product_I', 'desc_I'])

# find weighted sum based on share of make table sales that we calculated
iomatrix_long_byproduct['weightTimesDeltaValue'] = iomatrix_long_byproduct['value'] * iomatrix_long_byproduct['share_of_sales_by_industry']

# fully collapse to product level
iomatrix_long_byproduct = iomatrix_long_byproduct[['product_I', 'value', 'product_O']].groupby(['product_I', 'product_O'], as_index=False).sum(min_count=1)

# here's a square version
iomatrix_wide_byproduct = iomatrix_long_byproduct.pivot_table(index='product_I', columns='product_O', values='value', aggfunc='mean')

In [10]:
# make sure that we have the same products in all the data

# first, i need products that actually have a full series so we can run the VARs
# total number of unique dates
total_dates = beadata['date'].nunique()
# group by product and count the number of unique dates for each product
product_dates_count = beadata.dropna().groupby('product')['date'].nunique()
# filter products that appear at all dates
products_appear_all_dates = product_dates_count[product_dates_count == total_dates].index.tolist()

# next i deal with the I-O matrix products...
# list of buyers (the whole usetable_wide_fillna.sum(axis=1/axis=0)[index] != 0 thing is so that i dont get any division by 0)
inputproducts = [index for index in iomatrix_wide_byproduct.index if iomatrix_wide_byproduct.sum(axis=1)[index] != 0 and iomatrix_wide_byproduct.sum(axis=0)[index] != 0]
# list of sellers
outputproducts = [column for column in iomatrix_wide_byproduct.columns if iomatrix_wide_byproduct.sum(axis=1)[column] != 0 and iomatrix_wide_byproduct.sum(axis=0)[column] != 0]

# final list of products
products_to_include = list(set(beadata['product']) & set(iomatrix_wide_byproduct.index) & set(iomatrix_wide_byproduct.columns))
products_to_include.sort()

# filter for products based on this list
iomatrix_wide_byproduct = iomatrix_wide_byproduct[products_to_include]
iomatrix_wide_byproduct = iomatrix_wide_byproduct.loc[iomatrix_wide_byproduct.index.isin(products_to_include)]
beadata = beadata.loc[beadata['product'].isin(products_to_include)]

In [23]:
# then this sum gives the share, for each sector, of what is used in intermediates
intermediate_salesshares = iomatrix_wide_byproduct.sum(axis=1)

# now get intermediate cost shares...
intermediate_costshares = (iomatrix_wide_byproduct @ np.diag(1/intermediate_salesshares)).T

# fix index names and stuff which is annoying
intermediate_costshares.index = intermediate_salesshares.index
intermediate_salesshares = intermediate_salesshares.rename_axis('product')

In [24]:
# so..... with the way that we are using the requirements matrix
# to do all our calculations now i am still noticing all these sectors 
# that have negative value added

intermediate_salesshares[intermediate_salesshares > 1]

product
Accessories and parts                                              12.126869
Air transportation                                                  1.025057
Alcohol in purchased meals                                          6.305685
Amusement parks, campgrounds, and related recreational services     4.288247
Audio equipment                                                     2.285304
                                                                     ...    
Trust, fiduciary, and custody activities                            1.358800
Veterinary and other services for pets                              1.165749
Video and audio streaming and rental                                2.999380
Video discs, tapes, and permanent digital downloads                 1.085186
Women's and girls' clothing                                         4.491039
Length: 103, dtype: float64

In [13]:
# get separate series for prices, quantities, expenditures to save
prices = beadata[['product', 'date', 'priceindex']]
quantities = beadata[['product', 'date', 'quantityindex']]
expenditures = beadata[['product', 'date', 'expenditures']]
# prices.to_pickle(path_cleandata + 'inversions//prices.pkl')
# quantities.to_pickle(path_cleandata + 'inversions//quantities.pkl')
# expenditures.to_pickle(path_cleandata + 'inversions//expenditures.pkl')

In [14]:
# all time periods

# i want to get dates where there is data for everything
dates = list(set(prices['date'].unique()) & set(quantities['date'].unique()) & set(expenditures['date'].unique()))
dates.sort()

# choose one date per decade for the troubleshooting plots (maybe just the fifth year, July 31 per decade since its the middle of the year ish)
plotdates = [dt.datetime(1965,7,31), dt.datetime(1975,7,31), dt.datetime(1985,7,31), dt.datetime(1995,7,31), dt.datetime(2005,7,31), dt.datetime(2015,7,31)]

In [16]:
# calculate sales in product and time period
sales = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'sales': pd.Series(dtype='float')})
for date in dates:
    # filter expenditures for the current date
    expenditures_date = expenditures[expenditures['date'] == date][['product', 'expenditures']].set_index('product')
    expenditures_date = expenditures_date.sort_index()

    # Create the diagonal matrix from intermediate_salesshares
    diag_matrix = np.diag(intermediate_salesshares)
    
    # calculate sales for each product
    # intermediate_costshares.T @ diag(intermediate_salesshares) gets you the 2017 I-O matrix 2017 (inputs needed from one product to another to produce one unit of output)
    # we're assuming the same I-O matrix for each time period, so now we want sales in each time period using this I-O matrix!
    # (inputs needed from one product to another to produce one unit of output) x (expenditures for each product at time t) gets you sales at time t for each product
    sales_date = np.linalg.inv(np.identity(len(intermediate_costshares)) - (intermediate_costshares.T @ diag_matrix)) @ expenditures_date

    # set some columns to append
    sales_date['date'] = date
    sales_date['product'] = expenditures_date.index
    sales_date.rename(columns={'expenditures': 'sales'}, inplace=True)
    # append
    sales = pd.concat([sales, sales_date], ignore_index=True)

# sales.to_pickle(path_cleandata + 'inversions//sales.pkl')

In [ ]:
# calculate prices of intermediates (we use cobb douglas production with intermediates)
intermediates = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'intermediates': pd.Series(dtype='float')})
for date in dates:
    # filter prices for the current date
    prices_date = prices[prices['date'] == date][['product', 'priceindex']].set_index('product')
    prices_date = prices_date.sort_index()
    
    for i in products_to_include:
        intermediates.loc[len(intermediates)] = [date, i , np.exp(intermediate_costshares.loc[i] @ np.log(prices_date['priceindex']))]

# intermediates.to_pickle(path_cleandata + 'inversions//intermediateprices.pkl')

In [ ]:
# intermediate prices vs output prices (plot)

compare = pd.merge(left=intermediates, right=prices, on=['date', 'product'], how='inner')
compare = compare[compare['date'].isin(dates)].sort_values(['date', 'priceindex'], ascending=False)

for idx, date in enumerate(plotdates):
    toplot_date = compare[compare['date'] == date].set_index('product')
    products = toplot_date.index
    price = np.log10(toplot_date['priceindex'])
    intermediates_forplot = np.log10(toplot_date['intermediates'])
    diff = price - intermediates_forplot

    datestr = date.strftime("%Y/%m/%d").replace('/', '')

    # plots for price residuals

    # calculate heights for stacked bars
    positive_diff = np.where((diff >= 0) & (intermediates_forplot >= 0), diff, 0)
    positive_adjusted = np.where((diff >= 0) & (intermediates_forplot >= 0), intermediates_forplot, intermediates_forplot)
    negative_diff = np.where((diff < 0) & (intermediates_forplot < 0), diff, 0)
    negative_adjusted = np.where((diff < 0) & (intermediates_forplot < 0), intermediates_forplot, intermediates_forplot)

    # open figure
    fig, ax = plt.subplots(figsize=(60, 30))

    ax.axhline(y=0, color='black')

    # layering bars for the right display
    ax.bar(products, diff, color='salmon', label='Difference between price indexes and prices of intermediates')
    ax.bar(products, positive_adjusted, color='skyblue', label='Intermediates')
    ax.bar(products, positive_diff, color='salmon', bottom=positive_adjusted, label='_')
    ax.bar(products, negative_adjusted, color='skyblue', label='_')
    ax.bar(products, negative_diff, color='salmon', bottom=negative_adjusted, label='_')

    line, = ax.plot(products, price, linestyle='-', marker='o', color='black', linewidth=4, label='Price index')
    
    # set appearances
    ax.set_title('Prices, intermediates ' + datestr)
    ax.legend(loc='upper right')
    ax.set_ylabel('(log10) Values')
    ax.set_xticks(np.arange(len(products))) 
    ax.set_xticklabels(products, rotation=90)
    ax.set_xlim(-1, len(products))
    fig.tight_layout()

    # plt.savefig(path_figures + 'prices\\log_intermediates_' + datestr + '.pdf', bbox_inches='tight')

    plt.show()

In [ ]:
# intermediate prices vs output prices (plot, non-log scale)

for idx, date in enumerate(plotdates):
    toplot_date = compare[compare['date'] == date].set_index('product')
    products = toplot_date.index
    price = toplot_date['priceindex']
    intermediates_forplot = toplot_date['intermediates']
    diff = price - intermediates_forplot

    datestr = date.strftime("%Y/%m/%d").replace('/', '')

    # plots for price residuals

    # calculate heights for stacked bars
    positive_diff = np.where((diff >= 0) & (intermediates_forplot >= 0), diff, 0)
    positive_adjusted = np.where((diff >= 0) & (intermediates_forplot >= 0), intermediates_forplot, intermediates_forplot)
    negative_diff = np.where((diff < 0) & (intermediates_forplot < 0), diff, 0)
    negative_adjusted = np.where((diff < 0) & (intermediates_forplot < 0), intermediates_forplot, intermediates_forplot)

    # open figure
    fig, ax = plt.subplots(figsize=(60, 30))

    ax.axhline(y=0, color='black')

    # layering bars for the right display
    ax.bar(products, diff, color='salmon', label='Difference between price indexes and prices of intermediates')
    ax.bar(products, positive_adjusted, color='skyblue', label='Intermediates')
    ax.bar(products, positive_diff, color='salmon', bottom=positive_adjusted, label='_')
    ax.bar(products, negative_adjusted, color='skyblue', label='_')
    ax.bar(products, negative_diff, color='salmon', bottom=negative_adjusted, label='_')

    line, = ax.plot(products, price, linestyle='-', marker='o', color='black', linewidth=4, label='Price index')
    
    # set appearances
    ax.set_title('Prices, intermediates ' + datestr)
    ax.legend(loc='upper right')
    ax.set_ylabel('Values')
    ax.set_xticks(np.arange(len(products))) 
    ax.set_xticklabels(products, rotation=90)
    ax.set_xlim(-1, len(products))
    fig.tight_layout()

    # plt.savefig(path_figures + 'prices\\intermediates_' + datestr + '.pdf', bbox_inches='tight')

    plt.show()

In [ ]:
# calculate prices of value added
value_added = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'value_added': pd.Series(dtype='float')})
for date in dates:
    # filter prices for the current date
    prices_date = prices[prices['date'] == date][['product', 'priceindex']].set_index('product')
    prices_date = prices_date.sort_index()

    # filter intermediates for the current date
    intermediates_date = intermediates[intermediates['date'] == date][['product', 'intermediates']].set_index('product')
    intermediates_date = intermediates_date.sort_index()

    value_added_date = np.exp((1/(1 - intermediate_salesshares))*(np.log(prices_date['priceindex']) - intermediate_salesshares * np.log(intermediates_date['intermediates'])))

    value_added_date = value_added_date.reset_index().rename(columns={0: 'value_added'})
    value_added_date['date'] = date

    value_added = pd.concat([value_added, value_added_date], ignore_index=True)

# value_added.to_pickle(path_cleandata + 'inversions//valueaddedprices.pkl')

In [ ]:
# value added prices vs output prices (plot)

compare = pd.merge(left=value_added, right=prices, on=['date', 'product'], how='inner')
compare = compare[compare['date'].isin(dates)].sort_values(['date', 'priceindex'], ascending=False)

for idx, date in enumerate(plotdates):
    toplot_date = compare[compare['date'] == date].set_index('product')
    products = toplot_date.index
    price = np.log10(toplot_date['priceindex'])
    value_added_forplot = np.log10(toplot_date['value_added'])
    diff = price - value_added_forplot

    datestr = date.strftime("%Y/%m/%d").replace('/', '')

    # plots for price residuals

    # calculate heights for stacked bars
    positive_diff = np.where((diff >= 0) & (value_added_forplot >= 0), diff, 0)
    positive_adjusted = np.where((diff >= 0) & (value_added_forplot >= 0), value_added_forplot, value_added_forplot)
    negative_diff = np.where((diff < 0) & (value_added_forplot < 0), diff, 0)
    negative_adjusted = np.where((diff < 0) & (value_added_forplot < 0), value_added_forplot, value_added_forplot)

    # open figure
    fig, ax = plt.subplots(figsize=(60, 30))

    ax.axhline(y=0, color='black')

    # layering bars for the right display
    ax.bar(products, diff, color='salmon', label='Difference between price indexes and value added')
    ax.bar(products, positive_adjusted, color='skyblue', label='Value added')
    ax.bar(products, positive_diff, color='salmon', bottom=positive_adjusted, label='_')
    ax.bar(products, negative_adjusted, color='skyblue', label='_')
    ax.bar(products, negative_diff, color='salmon', bottom=negative_adjusted, label='_')

    line, = ax.plot(products, price, linestyle='-', marker='o', color='black', linewidth=4, label='Price index')
    
    # set appearances
    ax.set_title('Prices, value added ' + datestr)
    ax.legend(loc='upper right')
    ax.set_ylabel('(log10) Values')
    ax.set_xticks(np.arange(len(products))) 
    ax.set_xticklabels(products, rotation=90)
    ax.set_xlim(-1, len(products))
    fig.tight_layout()

    # plt.savefig(path_figures + 'prices\\log_valueadded_' + datestr + '.pdf', bbox_inches='tight')

    plt.show()